# Milestone 2
This notebook contains the classifier code for MS 2. For more info, see for example: [the sklearn docs](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html#training-a-classifier)

In [ ]:
from utils import get_data
import utils
from sklearn import metrics
from sklearn.pipeline import Pipeline

X_train, y_train, X_test, y_test = get_data()

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

def train_and_eval(clf: BaseEstimator,
                   gs_params: dict,
                   X_train: list[str],
                   y_train: list[int],
                   X_test: list[str],
                   y_test: list[int],) -> str:
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', clf),
    ])

    if gs_params:
        # do gridsearch if params are provided
        text_clf = GridSearchCV(text_clf, gs_params, cv=5, n_jobs=-1)

    text_clf.fit(X_train, y_train)
    y_pred = text_clf.predict(X_test)

    for param_name in sorted(gs_params.keys()):
        print("%s: %r" % (param_name, text_clf.best_params_[param_name]))

    return metrics.classification_report(y_test, y_pred, target_names=['no stress', 'stress']), y_test, y_pred


def analyze_results(y_test, y_pred):
    df_test = utils.get_full_test_data()
    false_negatives_sentiment_scores = []
    false_positives_sentiment_scores = []
    true_negatives_sentiment_scores = []
    true_positives_sentiment_scores = []

    false_negatives_lex_liwc_Tone = []
    false_positives_lex_liwc_Tone = []
    true_negatives_lex_liwc_Tone = []
    true_positives_lex_liwc_Tone = []

    false_negatives = [i for i, (actual, pred) in enumerate(zip(y_test, y_pred)) if actual == 1 and pred == 0]
    false_positives = [i for i, (actual, pred) in enumerate(zip(y_test, y_pred)) if actual == 0 and pred == 1]
    true_negatives = [i for i, (actual, pred) in enumerate(zip(y_test, y_pred)) if actual == 0 and pred == 0]
    true_positives = [i for i, (actual, pred) in enumerate(zip(y_test, y_pred)) if actual == 1 and pred == 1]

    print("\nFalse Negatives:")
    for count, index in enumerate(false_negatives):
        false_negatives_sentiment_scores.append(df_test.loc[index, 'sentiment'])
        false_negatives_lex_liwc_Tone.append(df_test.loc[index, 'lex_liwc_Tone'])
        if count <= 5:
            print(f'Actual: Stress, Predicted: No Stress, Sentiment: {df_test.loc[index, "sentiment"]}, Tone: {df_test.loc[index, "lex_liwc_Tone"]}, Text: {df_test.loc[index, "text"]}')

    print("\nFalse Positives:")
    for count, index in enumerate(false_positives):
        false_positives_sentiment_scores.append(df_test.loc[index, 'sentiment'])
        false_positives_lex_liwc_Tone.append(df_test.loc[index, 'lex_liwc_Tone'])
        if count <= 5:
            print(f'Actual: No Stress, Predicted: Stress, Sentiment: {df_test.loc[index, "sentiment"]}, Tone: {df_test.loc[index, "lex_liwc_Tone"]}, Text: {df_test.loc[index, "text"]}')

    for count, index in enumerate(true_negatives):
        true_negatives_sentiment_scores.append(df_test.loc[index, 'sentiment'])
        true_negatives_lex_liwc_Tone.append(df_test.loc[index, 'lex_liwc_Tone'])

    for count, index in enumerate(true_positives):
        true_positives_sentiment_scores.append(df_test.loc[index, 'sentiment'])
        true_positives_lex_liwc_Tone.append(df_test.loc[index, 'lex_liwc_Tone'])

    plot_distribution(false_negatives_sentiment_scores, false_positives_sentiment_scores, true_negatives_sentiment_scores, true_positives_sentiment_scores, 'Sentiment')
    plot_distribution(false_negatives_lex_liwc_Tone, false_positives_lex_liwc_Tone, true_negatives_lex_liwc_Tone, true_positives_lex_liwc_Tone, 'Tone')


def plot_distribution(false_negatives, false_positives, true_negatives, true_positives, description):
    data = [false_negatives, false_positives, true_negatives, true_positives]
    labels = ['False Negatives', 'False Positives', 'True Negatives', 'True Positives']

    plt.figure(figsize=(10, 6))
    sns.violinplot(data=data, palette=['red', 'purple', 'blue', 'green'], inner="quartile")
    plt.xticks(range(len(labels)), labels)
    plt.title(f'{description} Distribution for Misclassified Examples')
    plt.xlabel('Categories')
    plt.ylabel(f'{description}')
    plt.show()

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

res = train_and_eval(MultinomialNB(),
                     {},
                     X_train,
                     y_train,
                     X_test,
                     y_test)
print(res[0])
analyze_results(res[1], res[2])

### Analysis of result:
- The classifier has a very low number of false negatives but also a very high number of false positives.
- The classifier has a high recall, meaning it detects instances of stress very well.
- The overall accuracy is 65%, which is not good when 50% is random chance on a balanced dataset.
- When comparing the sentiment score between false negatives and false positives, we cannot observe any difference.
- Also, the tone seems very similar. However, when comparing the tone between the true negatives and true positives, we can see a different pattern, which is already a hint that tone is a good feature.

## RidgeClassifier

In [ ]:
from sklearn.linear_model import RidgeClassifier

res = train_and_eval(RidgeClassifier(tol=1e-2, solver="sparse_cg"),
                     {},
                     X_train,
                     y_train,
                     X_test,
                     y_test)
print(res[0])
analyze_results(res[1], res[2])

### Analysis of result:
- The Ridge Classifier has a more balanced trade-off between precision and recall compared to the previous classifier. Both precision and recall are around 70%, making a more balanced prediction.
- The overall accuracy has improved to 70%.
- The sentiment distribution is the same between false negatives and false positives.
- False positive classified examples have the tendency to have a nicer tone than false negatives.

## SVM

In [ ]:
from sklearn.linear_model import SGDClassifier

res = train_and_eval(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None),
                     {},
                     X_train,
                     y_train,
                     X_test,
                     y_test)
print(res[0])
analyze_results(res[1], res[2])

### Analysis of result:
- The SVM classifier shows precision and recall for stress detection at 71% and 77%, which again indicates a balanced performance.
- The overall accuracy is 72%, very similar to Ridge.
- The sentiment distribution is the same between false negatives and false positives.
- False positive classified examples have the tendency to have a nicer tone than false negatives.

## SVM __with__ Grid Search tuning

In [ ]:
from sklearn.linear_model import SGDClassifier

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__use_idf': (True, False),
    'clf__loss': ('hinge', 'perceptron'),
    'clf__alpha': (1e-1, 1e-2, 1e-3, 1e-4),
}

res = train_and_eval(SGDClassifier(),
                     parameters,
                     X_train,
                     y_train,
                     X_test,
                     y_test)
print(res[0])
analyze_results(res[1], res[2])

### Analysis of result:
- The SVM model maintains consistent precision (71%), recall (77%), and F1-score (74%).
- The model shows balanced performance in terms of true positives, true negatives, false positives, and false negatives.
- The distribution of tone and sentiment has not changed from the last SVM model, nor has the accuracy.